In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
import h5py

import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

2024-04-15 10:57:22.105231: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-15 10:57:22.161499: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 10:57:22.161526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 10:57:22.162418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-15 10:57:22.167884: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score



with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
    ls = list(hdf.keys())
    images = hdf.get('images')
    boxes = hdf.get('boxes')
    images = np.array(images)
    boxes = np.array(boxes)


image_normalized = (images + 1e-9) / 9.26
normalized_boxes = boxes / [1, 64, 64, 64, 64]

images_np = image_normalized

probabilities = np.array(normalized_boxes[:, :, :-4])
probabilities = tf.expand_dims(probabilities, axis=1)
boxes_np = np.array(normalized_boxes[:, :, 1:])
boxes_np = tf.expand_dims(boxes_np, axis=1)
batch_size = 128
dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)

input_shape = (64,64,1)
num_classes = 280
num_coordinates = 4

x_input = layers.Input(shape=input_shape)
#Layer 1
x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) 
x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
#Layer 2
x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
#Layer 3
x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
#Layer 4
x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
#Layer 5
x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) 


x = layers.Flatten()(x)
# Probability output
x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)

# Bounding box output
x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)


model = tf.keras.models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
model.compile(optimizer= optimizer, loss= {'x_prob_reshape': tf.keras.losses.BinaryCrossentropy(), 'x_boxes_reshape':tf.keras.losses.MeanSquaredError()}, metrics={'x_prob_reshape': 'accuracy'} )
# num_epochs = 100


# Load the trained model
loaded_model = tf.keras.models.load_model("/home/m3-learning/Documents/Research Data/Derrick's Object Detection/Models/M11overfittedmodel3variant.h5")

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((images_np,{'x_prob_reshape':probabilities,'x_boxes_reshape':boxes_np}))
dataset = dataset.batch(2000)
#dataset = dataset.shuffle(buffer_size=1000, reshuffle_each_iteration=True) 
inputs,targets = next(iter(dataset))
output = loaded_model.predict(inputs)

2024-04-15 10:58:30.466165: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-15 10:58:30.582529: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


15/63 [======>.......................] - ETA: 0s

2024-04-15 10:58:30.910137: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


63/63 [==============================] - 1s 11ms/step


In [5]:

# Example function to evaluate predictions
def evaluate_predictions(predictions, ground_truth):
    predicted_classes = np.argmax(predictions[0], axis=-1)
    true_classes = np.argmax(ground_truth['x_prob_reshape'], axis=-1)
    accuracy = accuracy_score(true_classes.flatten(), predicted_classes.flatten())
    
    # Additional metrics can be calculated here, such as IoU for bounding boxes
    return accuracy

#Evaluate the model
accuracy = evaluate_predictions(output, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np})

# Save results
np.savez('evaluation_results.npz', accuracy=accuracy, predictions=output)

print(f"Model evaluation completed with accuracy: {accuracy}")


ValueError: Found input variables with inconsistent numbers of samples: [2800000, 560000]

In [ ]:
with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
    ls = list(hdf.keys())
    images = hdf.get('images')
    boxes = hdf.get('boxes')
    images = np.array(images)
    boxes = np.array(boxes)


image_normalized = (images + 1e-9) / 9.26
normalized_boxes = boxes / [1, 64, 64, 64, 64]

In [ ]:
model  = tf.keras.models.load_model("/home/m3-learning/Documents/Research Data/Derrick's Object Detection/Models/M11overfittedmodel3variant.h5")

In [6]:
import tensorflow as tf

def load_model(model_path):
    return tf.keras.models.load_model(model_path)


In [7]:
def preprocess_data(images, labels):
    # Implement the specific preprocessing steps you used during training here
    processed_images = images / 255.0  # Example normalization
    return processed_images, labels


In [8]:
def predict(model, test_images):
    predictions = model.predict(test_images)
    return predictions


In [9]:
from sklearn.metrics import mean_absolute_error, precision_score, recall_score, confusion_matrix

def calculate_metrics(predictions, ground_truth):
    # Threshold predictions if necessary (for example, for binary classification)
    predicted_labels = (predictions > 0.5).astype(int)
    mae = mean_absolute_error(ground_truth, predicted_labels)
    precision = precision_score(ground_truth, predicted_labels)
    recall = recall_score(ground_truth, predicted_labels)
    
    # Calculate additional metrics as needed
    return mae, precision, recall


In [ ]:
def main():
    model_path = 'path_to_your_model.h5'
    test_images, test_labels = load_test_data('path_to_your_test_data')
    processed_test_images, test_labels = preprocess_data(test_images, test_labels)

    model = load_model(model_path)
    predictions = predict(model, processed_test_images)

    mae, precision, recall = calculate_metrics(predictions, test_labels)
    save_results(mae, precision, recall, 'path_to_save_results.npz')

    print("Evaluation completed. Results saved.")

if __name__ == "__main__":
    main()
